# Block Detail
In this notebook, I extract block details given a start_block number and end_block number. Information extracted include:
- block detail
- gas usage
- timestamp
- sibling blocks

In [1]:
import os
from hexbytes import HexBytes
import csv
import numpy as np
import sys
import copy
import web3_api

## Data Extraction
One possible way to extract data in an orderly fashion is first to store the transaction hash in a database. Then using
this database to extract detailed information about the block and transactions included in the block.

In [2]:
block_start = 13500000
block_end = 15850000
database = {} # store transaction hash with timestamp
blockdetail = {} # store block details

## Arbitrum Blockchain Explorer
Source: https://arbiscan.io/

There are three different types of data we can extract from Arbitrum:
1. Arbitrum block details
2. Arbitrum transaction details
3. L2 --> L1 transactions
    - L1 block number included this batch of transactions
    - L1 transaction hash
    - L2 transaction hash

In [4]:
# Let's try some data extraction from Arbitrum blockchain.
block_info = web3_api.get_block_info(34521828, detail=False)
block_info

AttributeDict({'baseFeePerGas': 100000000,
 'difficulty': 1,
 'extraData': HexBytes('0x1557a299040487b8ba832954adf793d0d264c7589ef8a3283cccba9897b75eff'),
 'gasLimit': 1125899906842624,
 'gasUsed': 306571,
 'hash': HexBytes('0xcb71c95a2aa8a853085ff82a5b5d49e0ccffe8529a3330ed5d72c7c07dbe28cf'),
 'l1BlockNumber': '0xf24331',
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000001000000000000000000000000000400000000000000000000000001000000000000000000000000010000001000000000000000000000000000000000000000000000000000020000000000000000000200000000000000000000000000000000000000000001000020000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000010000000'),
 'miner': '0xA4b000000000000000000073657175656e636572',
 'mixHash': HexBytes('0x0000000000002bfa0000000000f24331000000000000000700000

In [6]:
# Detailed block info
block_info = web3_api.get_block_info(34521828, detail=True)
block_info

AttributeDict({'baseFeePerGas': 100000000,
 'difficulty': 1,
 'extraData': HexBytes('0x1557a299040487b8ba832954adf793d0d264c7589ef8a3283cccba9897b75eff'),
 'gasLimit': 1125899906842624,
 'gasUsed': 306571,
 'hash': HexBytes('0xcb71c95a2aa8a853085ff82a5b5d49e0ccffe8529a3330ed5d72c7c07dbe28cf'),
 'l1BlockNumber': '0xf24331',
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000001000000000000000000000000000400000000000000000000000001000000000000000000000000010000001000000000000000000000000000000000000000000000000000020000000000000000000200000000000000000000000000000000000000000001000020000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000010000000'),
 'miner': '0xA4b000000000000000000073657175656e636572',
 'mixHash': HexBytes('0x0000000000002bfa0000000000f24331000000000000000700000

In [11]:
# Extract transaction receipt for an L2 transaction
txhash = HexBytes('0x38b26e3e404eb9cd6f01d7cf7b688a58885ed1d6c06ef76a042ff9102f53ea60')
tx_receipt = web3_api.get_tx_receipt(txhash)
tx_info = web3_api.get_tx_info(txhash)
tx_receipt, '*'*100, tx_info

(AttributeDict({'blockHash': HexBytes('0x4e9232174956e32a521859a2a2470734d317d2f03f4a9b5b87819a9b3c511441'),
  'blockNumber': 34521831,
  'contractAddress': None,
  'cumulativeGasUsed': 849301,
  'effectiveGasPrice': 100000000,
  'from': '0xB149f353383aB92e74c4cbA3fEB848b04bfB3617',
  'gasUsed': 849301,
  'gasUsedForL1': '0x9bd05',
  'l1BlockNumber': '0xf24331',
  'logs': [AttributeDict({'address': '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1',
    'blockHash': HexBytes('0x4e9232174956e32a521859a2a2470734d317d2f03f4a9b5b87819a9b3c511441'),
    'blockNumber': 34521831,
    'data': '0x00000000000000000000000000000000000000000000000002a00c68e806ae77',
    'logIndex': 0,
    'removed': False,
    'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
     HexBytes('0x000000000000000000000000c31e54c7a869b9fcbecc14363cf510d1c41fa443'),
     HexBytes('0x00000000000000000000000068b3465833fb72a70ecdf485e0e4c7bd8665fc45')],
    'transactionHash': HexBytes('0x38

In [12]:
# Extract data from an L2 --> L1 transaction
txhash = HexBytes('0xe02152fb726c3f4b38d9e9d5f7c4d260a03cc8752f337081c133f54e4929ecdf')
tx_receipt = web3_api.get_tx_receipt(txhash)
tx_info = web3_api.get_tx_info(txhash)
tx_receipt, '*'*100, tx_info

(AttributeDict({'blockHash': HexBytes('0xda72d5c6e1260fcf235a1ae0dd70bf4bf420fd1e63db49065b3aef9c9b7cc03c'),
  'blockNumber': 34530399,
  'contractAddress': None,
  'cumulativeGasUsed': 0,
  'effectiveGasPrice': 100000000,
  'from': '0x9F45B74a442149Da5DcE497E9278a0ae33f8D8a1',
  'gasUsed': 0,
  'gasUsedForL1': '0x0',
  'l1BlockNumber': '0xf24414',
  'logs': [],
  'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
  'status': 1,
  'to': '0x8E34B74A442149DA5Dce497E9278a0Ae33f8C790',
  'transactionHash': He